In [10]:
import pandas as pd
import re
from datetime import datetime
from datetime import timedelta

In [11]:
data = pd.read_csv('trades.csv',sep=',')

In [12]:
data.head()

,MS US Equity,Unnamed: 1,Unnamed: 2,TXN US Equity,Unnamed: 4
0,Dates,Close,NaN,Dates,Close
1,23/1/2019 9:30,42.57,NaN,23/1/2019 9:30,96.63
2,23/1/2019 9:31,42.46,NaN,23/1/2019 9:31,96.42
3,23/1/2019 9:32,42.49,NaN,23/1/2019 9:32,96.51
4,23/1/2019 9:33,42.43,NaN,23/1/2019 9:33,96.81


In [13]:
equity = 'TXN'
ticker = equity + ' US Equity'
index=0
counter=0

for col in data.columns:
    if col==ticker: 
        index = counter
    counter = counter+1

In [14]:
index

3

In [15]:
formatDateTime = '%d/%m/%Y %H:%M'

In [16]:
#Settings
startTime = '9:30'
endTime = '16:00'
startDate = '1/2/2019'
endDate = '10/2/2019'

minutesDelta = 30

In [20]:
startDateTime = datetime.strptime(startDate+' '+startTime, formatDateTime)
endDateTime = datetime.strptime(endDate+' '+endTime, formatDateTime)

dateTime = startDateTime
counter = 1

while dateTime<=endDateTime:
    counter = counter + 1
    if data.iloc[counter,index] != '' and data.iloc[counter,index] != 'nan':
        #print(data.iloc[counter,index])
        recordDateTime = datetime.strptime(data.iloc[counter,index], formatDateTime)
        if recordDateTime>=dateTime:
            if recordDateTime == dateTime:
                dateTime = dateTime + timedelta(minutes=minutesDelta)
            else:
                dateTime = dateTime + timedelta(days=1)
                #print(str(dateTime.day) +"/"+ str(dateTime.month) + "/"+ str(dateTime.year) +' '+startTime)
                dateTime = datetime.strptime(str(dateTime.day) +"/"+ str(dateTime.month) + "/"+ str(dateTime.year) +' '+startTime, formatDateTime)
            print(recordDateTime)
            print(data.iloc[counter, index+1])

2019-02-01 09:30:00
100.46
2019-02-01 10:00:00
100.89
2019-02-01 10:30:00
101.96
2019-02-01 11:00:00
101.18
2019-02-01 11:30:00
101.56
2019-02-01 12:00:00
101.73
2019-02-01 12:30:00
101.43
2019-02-01 13:00:00
101.26
2019-02-01 13:30:00
101.7
2019-02-01 14:00:00
101.65
2019-02-01 14:30:00
101.45
2019-02-01 15:00:00
101.21
2019-02-01 15:30:00
101.53
2019-02-01 16:29:00
101.99
2019-02-04 09:30:00
101.89
2019-02-04 09:31:00
101.65
2019-02-04 09:32:00
102.035
2019-02-05 09:30:00
101.865
2019-02-05 10:00:00
101.17
2019-02-05 10:30:00
101.35
2019-02-05 11:00:00
101.605
2019-02-05 11:30:00
101.564
2019-02-05 12:00:00
101.72
2019-02-05 12:30:00
101.46
2019-02-05 13:00:00
101.48
2019-02-05 13:30:00
101.7
2019-02-05 14:00:00
102.03
2019-02-05 14:30:00
102.27
2019-02-05 15:00:00
102.34
2019-02-05 15:30:00
102.3
2019-02-05 16:29:00
102.32
2019-02-06 09:30:00
104.92
2019-02-06 10:00:00
105.94
2019-02-06 10:30:00
105.51
2019-02-06 11:00:00
105.98
2019-02-06 11:30:00
106.166
2019-02-06 12:00:00
106.02